In [1]:
from itext2kg import iText2KG
from langchain.document_loaders import PyPDFLoader
from itext2kg.documents_distiller import DocumentsDistiller, Article
from pydantic import BaseModel, Field
from typing import List, Tuple
from itext2kg.graph_integration import GraphIntegrator
from langchain_ollama import ChatOllama, OllamaEmbeddings

d:\soft\anaconda3\envs\itext2kgenv\Lib\site-packages\pydantic\_internal\_generate_schema.py:502: UserWarning: <built-in function array> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [2]:
llm = ChatOllama(
    model="llama3",
    temperature=0,
)

embeddings = OllamaEmbeddings(
    model="llama3",
)

In [3]:
class ArticleResults(BaseModel):
    abstract:str = Field(description="Brief summary of the article's abstract")
    key_findings:str = Field(description="The key findings of the article")
    limitation_of_sota : str=Field(description="limitation of the existing work")
    proposed_solution : str = Field(description="the proposed solution in details")
    paper_limitations : str=Field(description="The limitations of the proposed solution of the paper")

In [4]:
documents_information = [
    ("../datasets/llm-tikg.pdf", [11,10], ArticleResults, 'scientific article'),
    ("../datasets/actionable-cyber-threat.pdf", [12,11,10], ArticleResults, 'scientific article')
]

In [5]:
def upload_and_distill(documents_information: List[Tuple[str, List[int], BaseModel]]):
    distilled_docs = []
    
    for path_, exclude_pages, blueprint, document_type in documents_information:
        
        loader = PyPDFLoader(path_)
        pages = loader.load_and_split()
        pages = [page for page in pages if page.metadata["page"]+1 not in exclude_pages] # Exclude some pages (unecessary pages, for example, the references)
        document_distiller = DocumentsDistiller(llm_model=llm)
        
        IE_query = f'''
        # DIRECTIVES : 
        - Act like an experienced information extractor.
        - You have a chunk of a {document_type}
        - If you do not find the right information, keep its place empty.
        '''
        
        # Distill document content with query
        distilled_doc = document_distiller.distill(
            documents=[page.page_content.replace("{", '[').replace("}", "]") for page in pages],
            IE_query=IE_query,
            output_data_structure=blueprint
        )
        
        # Filter and format distilled document results
        distilled_docs.append([
            f"{document_type}'s {key} - {value}".replace("{", "[").replace("}", "]") 
            for key, value in distilled_doc.items() 
            if value and value != []
        ])
    
    return distilled_docs

In [6]:
distilled_docs = upload_and_distill(documents_information=documents_information)

In [ ]:
itext2kg = iText2KG(llm_model = llm, embeddings_model = embeddings)

[INFO] ------- Extracting Entities from the Document 1
{'entities': [{'label': 'Attackers', 'name': 'Malware and Attackers'}, {'label': 'Malware', 'name': 'Identical Malware Techniques'}]}
[INFO] ------- Extracting Relations from the Document 1
{'relationships': [{'startNode': {'label': 'Malware', 'name': 'identical malware techniques'}, 'endNode': {'label': 'Attackers', 'name': 'malware and attackers'}, 'name': 'employs'}]}
[INFO] Verification of invented entities
[INFO] ------- Extracting Entities from the Document 2
{'entities': [{'label': 'Abstract', 'name': 'The summary of the scientific article'}, {'label': 'Key Findings', 'name': 'The main results and discoveries presented in the article'}, {'label': 'Limitation of SOTA', 'name': 'The limitations of the current state-of-the-art (SOTA) methods or techniques'}, {'label': 'Proposed Solution', 'name': 'The new approach, method, or technique proposed in the article to address the limitations of SOTA'}, {'label': 'Paper Limitations', 

ValueError: Failed to extract relationships after multiple attempts.

In [8]:
kg = itext2kg.build_graph(sections=distilled_docs[0], ent_threshold=0.7, rel_threshold=0.7)


[INFO] ------- Extracting Entities from the Document 1
{'entities': [{'label': 'Attackers', 'name': 'Malware and Attackers'}, {'label': 'Malware', 'name': 'Identical Malware Techniques'}]}
[INFO] ------- Extracting Relations from the Document 1
{'relationships': [{'startNode': {'label': 'Malware', 'name': 'identical malware techniques'}, 'endNode': {'label': 'Attackers', 'name': 'malware and attackers'}, 'name': 'employs'}]}
[INFO] Verification of invented entities
[INFO] ------- Extracting Entities from the Document 2
{'entities': [{'label': 'Abstract', 'name': 'The summary of the scientific article'}, {'label': 'Key Findings', 'name': 'The main discoveries or results presented in the article'}, {'label': 'Limitation of SOTA', 'name': 'The shortcomings or limitations of the current state-of-the-art (SOTA) methods or techniques'}, {'label': 'Proposed Solution', 'name': 'The new approach, method, or technique proposed in the article to address the limitations of SOTA'}, {'label': 'Paper

ValueError: Failed to extract relationships after multiple attempts.